In [64]:
with open("./challenge/output.txt", "r") as f:
    g = f.readline().split("=")[-1].strip()
    f.readline()
    A = f.readline().split("=")[-1].strip()
    f.readline()
    B = f.readline().split("=")[-1].strip()
    f.readline()
    c = f.readline().split("=")[-1].strip()
    g = eval(g)
    A = eval(A)
    B = eval(B)
    c = eval(c)

In [29]:
from sage.arith.functions import LCM_list
def convert_to_permutation(l):
    return Permutation(vector(l) + vector([1] * len(l)))

def permutation_pow(p, n):
    res = Permutation(vector(range(p.size())) + vector([1] * p.size()))
    while n != 0:
       res *= p
       n -= 1
    return res

def max_cycle_permutation(p:Permutation):
    cycles = p.cycle_tuples()
    num_cycles = list(map(lambda x: len(x), cycles))
    return LCM_list(num_cycles)

def permutation_pow_fast(g: Permutation, n):
    g_cycle: List[Tuple[int]] = g.cycle_tuples()
    ans = [0] * g.size()
    for cycle in g_cycle:
        p = n % len(cycle)
        for i in range(len(cycle)):
            ans[cycle[(i + p) % len(cycle)] - 1] = g[cycle[i] - 1]
    return Permutation(ans)

In [69]:
g = convert_to_matrix(g)
A = convert_to_matrix(A)
B = convert_to_matrix(B)

KeyboardInterrupt: 

In [2]:
g = [2, 4, 1, 0, 3]
mg = Matrix(len(g), len(g))
for i, _g in enumerate(g):
    mg[i, _g] = 1

rg = mg*mg*mg
discrete_log(rg, mg)

3

In [32]:
from random import shuffle
n = 15
g = list(range(n))
shuffle(g)
g = convert_to_permutation(g)
r = convert_to_permutation(list(range(n))) * g * g
ans = 1
while r != g:
    r *= g
    ans += 1
show(ans)
cycle = g.cycle_tuples()
print(list(range(1, n + 1)))
print(r)
r *=g
print(tuple(r))
print(r*g)
print(permutation_pow_fast(g,2))
print(cycle)

36

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[12, 10, 14, 1, 15, 2, 9, 6, 5, 8, 11, 7, 13, 4, 3]
(7, 8, 4, 12, 3, 10, 5, 2, 15, 6, 11, 9, 13, 1, 14)
[9, 6, 1, 7, 14, 8, 15, 10, 3, 2, 11, 5, 13, 12, 4]
[4, 6, 15, 14, 9, 8, 12, 10, 7, 2, 11, 1, 13, 3, 5]
[(1, 12, 7, 9, 5, 15, 3, 14, 4), (2, 10, 8, 6), (11,), (13,)]


In [19]:
from math import ceil, sqrt
from typing import Dict


class Permutation:
    def __init__(self, mapping):
        self.length = len(mapping)

        assert set(mapping) == set(
            range(self.length)
        )  # ensure it contains all numbers from 0 to length-1, with no repetitions
        self.mapping = list(mapping)

    def __hash__(self) -> int:
        return hash(tuple(self.mapping))

    def __call__(self, *args, **kwargs):
        idx, *_ = args
        assert idx in range(self.length)
        return self.mapping[idx]

    def __mul__(self, other):
        ans = []

        for i in range(self.length):
            ans.append(self(other(i)))

        return Permutation(ans)

    def inverse(self):
        ans = [None] * self.length

        for i in range(self.length):
            ans[self(i)] = i

        return Permutation(ans)

    def __pow__(self, power, modulo=None):
        ans = Permutation.identity(self.length)
        ctr = self

        while power > 0:
            if power % 2 == 1:
                ans *= ctr
            ctr *= ctr
            power //= 2

        return ans

    def __str__(self):
        return str(self.mapping)

    def __repr__(self) -> str:
        return self.__str__()

    def __eq__(self, __value: object) -> bool:
        assert isinstance(__value, Permutation)
        for x, y in zip(self.mapping, __value.mapping):
            if x != y:
                return False
        return True

    def copy(self):
        return Permutation(self.mapping.copy())

    def log(self, g: "Permutation"):  # Not Accurate
        n = ceil(sqrt(self.length))

        mul = g**n

        value: Dict[Permutation, int] = dict()
        cur = Permutation.identity(self.length)
        for i in range(n):
            if not value.get(cur):
                value[cur] = i
            cur *= mul

        cur = self.copy()
        for i in range(self.length):
            if value.get(cur):
                return n * value.get(cur) - i
            if cur == g:
                return self.length - i
            cur *= g

        return None

    @staticmethod
    def identity(length):
        return Permutation(range(length))

p = Permutation([1, 3, 5, 0, 2, 4,8,6,7])
print(p)
print(p.inverse())

[1, 3, 5, 0, 2, 4, 8, 6, 7]
[3, 0, 4, 1, 5, 2, 7, 8, 6]
